In [10]:
import unittest
import extractor_spark

class Testextract(unittest.TestCase):
    def test_extractor(self):
        input_file = '../xkcd2347/data_collection/Input_data/3k_valid_repos'
        output_file = '3krepodata_test'
        git_hub_key = 'github_token'
        hop = 1
        partitions = 2
        extractor_spark.getDependencies(input_file, output_file, git_hub_key, hop, partitions)    
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

C:\Users\Administrator\Downloads\SMA-master\test\extractor_spark.py:33: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  print("Error running spark job")
E

Error running spark job



ERROR: test_extractor (__main__.Testextract)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-10-6400ca55dc6f>", line 11, in test_extractor
    extractor_spark.getDependencies(input_file, output_file, git_hub_key, hop, partitions)
  File "C:\Users\Administrator\Downloads\SMA-master\test\extractor_spark.py", line 37, in getDependencies
    sc.stop()
UnboundLocalError: local variable 'sc' referenced before assignment

----------------------------------------------------------------------
Ran 1 test in 1.552s

FAILED (errors=1)


In [9]:
import xkcd2347
import json
from pyspark import SparkContext
from tqdm import tqdm
import sys

def getDependencies(input_file, output_file, git_hub_key, hop, partitions):
    dependency_data = {}
    no_dependency_data = []
    gitLink = []

    gh = xkcd2347.GitHub(key = git_hub_key)
    result = []

    def process_line(link):
        link = link.rstrip()
        owner, repo = link.split(".com/")[1].split("/")
        result = [dep['packageName'] for dep in gh.get_dependencies(repo_owner = owner, repo_name = repo, depth = hop)]
        if(len(result) > 0):
            return (link, result)
        else:
            return (link, ["No dependency"])
    try:
        sc = SparkContext("local[*]","PySpark Tutorial")
        result = sc.textFile(input_file, partitions).map(process_line).collect()
    except:
        print("Error running spark job")

    with open(output_file, 'w') as fp:
        json.dump(dict(result), fp, indent=4)
    #sc.stop()

if __name__ == "__main__":
    input_file = ' 3k_valid_repos'
    output_file = '3krepodata_test'
    git_hub_key = 'github_token'
    hop = 1
    partitions = 8
    getDependencies(input_file, output_file, git_hub_key, hop, partitions)

Error running spark job
